# Lab 3 : Diving Into Quantum Algorithms
***

## Section 1: QPE


<img src="./resources/qpe_tex_qz.png" alt="QPE Circuit" style="width: 800px;"/>


Here we will construct a set of functions to generate a Quantum Phase Estimation circuit.

*What will students do?*:
1. Generate a QPE circuit with an arbitrary number of counting qubits
    1. Give hint to start at 4 or other small number and modularize it
1. Use the QPE circuit to construct Shor's algorithm for small integers.
    

In [ ]:
import qiskit as qk
import numpy as np
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


In [ ]:
#QFT Circuit
def qft(n):
    """Creates an n-qubit QFT circuit"""
    circuit = qk.QuantumCircuit(n)
    def swap_registers(circuit, n):
        for qubit in range(n//2):
            circuit.swap(qubit, n-qubit-1)
        return circuit
    def qft_rotations(circuit, n):
        """Performs qft on the first n qubits in circuit (without swaps)"""
        if n == 0:
            return circuit
        n -= 1
        circuit.h(n)
        for qubit in range(n):
            circuit.cp(np.pi/2**(n-qubit), qubit, n)
        qft_rotations(circuit, n)
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

#Inverse Quantum Fourier Transform
def qft_dagger(qc, n):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)

### Step 1: Set up a QPE Circuit with four counting qubits



Let's pick a phase gate with $\theta = \frac{1}{3}$ as a simple example unitary to test creating a QPE circuit.  Here we'll use Qiskit's `PhaseGate` which applies $P\ket{1}=e^{i\lambda}\ket{1}$.  Since we want to examine QPE under a unitary with the form $U\ket{1}=e^{i2\pi \theta}$, we should set $\lambda=\frac{2\pi}{3}$.

Create a QPE circuit with four counting qubits and name the circuit `qpe4`.  It may be helpful to define two `QuantumRegister` objects, one for the "system" where the unitary will be applied and one for where the phase information will be stored.  Feel free to reference the Qiskit Textbook's chapter on [Quantum Phase Estimation](https://learn.qiskit.org/course/ch-algorithms/quantum-phase-estimation#getting_more_precision).

It should look something like this:

<img src="./resources/qpe4_circuit.png" alt="QPE 4 Phase Circuit" style="width: 1000px;"/>


In [ ]:

phase_register_size = 4
qpe4 = qk.QuantumCircuit(phase_register_size+1, phase_register_size)

### Insert your code here



Now use the `AerSimulator` to simulate this circuit and plot the histogram of the results.  Use 2000 shots.


In [ ]:
## Run this cell to simulate 'qpe4' and to plot the histogram of the result
sim = qk.Aer.get_backend('aer_simulator')
shots = 2000
count_qpe4 = qk.execute(qpe4, sim, shots=shots).result().get_counts()
plot_histogram(count_qpe4, figsize=(9,5))

In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex1 

grade_lab3_ex1(count_qpe4)

Next write a lil function to process the bit strings into the estimate of $\theta$.  Recall that the phase estimate is written in the form:

$$ \theta = 0.\theta_1\theta_2\theta_3...\theta_t = \frac{\theta_1}{2^1} + \frac{\theta_2}{2^2} + \frac{\theta_3}{2^3} + ... + \frac{\theta_t}{2^t} $$

where $\theta_i = \{0,1\}$.  What is the estimated phase?  To within what power of 2 is it accurate to? ($2^{-2}$, $2^{-3}$, $2^{-4}$, etc.)

In [ ]:

#Grab the highest probability measurement
max_binary_counts = 0
max_binary_val = ''
for key, item in count_qpe4.items():
    if item > max_binary_counts:
        max_binary_val = key

## Your function goes here



estimated_phase = # calculate the estimated phase
phase_accuracy = # calculate the phase accuracy



In [ ]:

from qc_grader.challenges.qgss_2023 import grade_lab3_ex2 

grade_lab3_ex2([estimated_phase, phase_accuracy])

## Step 2: Run on Noisy Hardware

Now run this circuit using your favorite backend!  Transpile this circuit a number of times (you pick how many) and pick the one with the lowest and highest circuit depth. 

Transpile the circuit with the parameter optimization_level = 3 to reduce the error in the result. Qiskit by default uses a stochastic swap mapper to place the needed SWAP gates, which varies the transpiled circuit results even under the same runtime settings. Therefore, to achieve shorter depth transpiled circuit for smaller error in the outcome, transpile qpe4 multiple times and choose one with the minimum circuit depth. Select the maximum circuit depth one as well to compare against.



In [ ]:
from qiskit_ibm_provider import IBMProvider


provider = IBMProvider()
hub = "YOUR_HUB"
group = "YOUR_GROUP"
project = "YOUR_PROJECT"

backend = provider.get_backend(backend_name, instance=f"{hub}/{group}/{project}")

# your code goes here



In [ ]:
shots = 2000

#Run the minimum depth qpe circuit
job_min_qpe4 = backend.run(min_depth_qpe, sim, shots=shots)
print(job_min_qpe4.job_id())

#Gather the count data
count_min_qpe4 = job_min_qpe4.result().get_counts()
plot_histogram(count_min_qpe4, figsize=(9,5))

In [ ]:

#Run the maximum depth qpe circuit
job_max_qpe4 = backend.run(max_depth_qpe, sim, shots=shots)
print(job_max_qpe4.job_id())

#Gather the count data
count_max_qpe4 = job_max_qpe4.result().get_counts()
plot_histogram(count_max_qpe4, figsize=(9,5))

## Step 3: Try with a different $\theta$

Now try the same procedure with $\theta = \frac{1}{5}$.  Rewrite your code written above to create a function which generates a QPE circuit with $n$ register qubits.  How many register qubits storing the phase information are needed for the estimate to be accurate to within $2^{-3}$? 

*Hint: It may be easier to iterate over different phase register sizes by creating a callable function. Perhaps call it* `qpe_circuit`

In [ ]:


def qpe_circuit(register_size):

    # Your code goes here
    
    
    return qpe
    



In [ ]:
## Run this cell to simulate 'qpe4' and to plot the histogram of the result
reg_size = # Vary the register sizes

qpe_check = qpe_circuit(reg_size)
sim = qk.Aer.get_backend('aer_simulator')
shots = 2000
count_qpe4 = qk.execute(qpe_check, sim, shots=shots).result().get_counts()
plot_histogram(count_qpe4, figsize=(9,5))

In [ ]:


# Process the count data to determine accuracy of the estimated phase





In [ ]:
required_register_size = #your answer here

In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex3

grade_lab3_ex3(required_register_size)

## Section 2: Shor's Algorithm
***

Here we will construct a set of functions to implement Shor's algorithm.  Remember that the goal of this algorithm is to find the prime factors of some large number $N$ and the key speedup this algorithm provides is by executing the period-finding part using a quantum computer.  This is where this section of the lab will focus.

*What will students do?*:
1. Construct a compiled version of Shor's algorithm

Recall the Shor's algorithm is composed of the following steps:
1. Choose a co-prime $a$, where $a\in [2,N-1]$ and the greatest common divisor of $a$ and $N$ is 1.
1. Find the order (periodicity) of $a$ modulo $N$, i.e. the smallest integer $r$ such that $a^r\text{mod} N=1$
1. Obtain the factor of $N$ by computing the greatest common divisor of $a^{r/2} \pm 1$ and $N$.

## Step 1. Period Finding

To begin, we'll use the unitary operator: $$ U\ket{y} \equiv \ket{ay\ \text{mod} N} $$


and explore the superposition state: 
$$
\ket{u} = \frac{1}{\sqrt{r}}\sum_{k=0}^{r-1} e^{-\frac{2\pi ik}{r}}\ket{a^k \text{mod}N} 
$$

Let's pick $a=3$ and $N=35$ as an example and investigate what the action of $U$ is on $\ket{u}$
\begin{align}
    U\ket{u} &= U\frac{1}{\sqrt{r}}\left( \ket{1} + e^{-\frac{2\pi i}{r}}\ket{3} + e^{\frac{-4\pi i}{r}}\ket{9} + ... + e^{\frac{-20\pi i}{r}}\ket{4} + e^{\frac{-22\pi i}{r}}\ket{12} \right) \\
    & =\frac{1}{\sqrt{r}}\left( U\ket{1} + e^{-\frac{2\pi i}{r}}U\ket{3} + e^{\frac{-4\pi i}{r}}U\ket{9} + ... + e^{\frac{-20\pi i}{r}}U\ket{4} + e^{\frac{-22\pi i}{r}}U\ket{12} \right) \\
    &= \frac{1}{\sqrt{r}}\left( \ket{3} + e^{-\frac{2\pi i}{r}}\ket{9} + e^{\frac{-4\pi i}{r}}\ket{27} + ... + e^{\frac{-20\pi i}{r}}\ket{12} + e^{\frac{-22\pi i}{r}}\ket{1} \right) \\
    &= \frac{e^{\frac{2\pi i}{r}}}{\sqrt{r}}\left( e^{-\frac{2\pi i}{r}}\ket{3} + e^{\frac{-4\pi i}{r}}\ket{9} + ... + e^{\frac{-20\pi i}{r}}\ket{4} + e^{\frac{-22\pi i}{r}}\ket{12} + \ket{1} \right) \\
    &= \frac{e^{\frac{2\pi i}{r}}}{\sqrt{r}} \ket{u}.
\end{align}


This is a particularly helpful eigenvalue as it contains $r$.  In fact, it needs to be included in order to ensure the phase differences between the basis states are equal.  This is also not the only eigenstate of $U$.  FOr us to generalize further, we can multiply an integer $s$ to each of these phases, which will then show up in our eigenvalue

\begin{align}
    \ket{u_s} &= \frac{1}{\sqrt{r}}\sum_{k=0}^{r-1} e^{\frac{-2\pi isk}{r}\ket{a^k\text{mod} N}} \\
    U\ket{u_s} &= e^{\frac{2\pi is}{r}}\ket{u_s}.
\end{align}


Now we have an eigenstate for each integer $0 \leq s \leq r$.  Notably, if we add up all of these eigenstates, the phases cancel all other basis states except $\ket{1}$ $$ \frac{1}{\sqrt{r}} \sum_{s=0}^{r-1}\ket{u_s} = \ket{1} $$.


Since any state in the computational basis can be written as a linear combination of these eigenstates, if we do QPE on $U$ using the state $\ket{1}$, we will measure a phase 

$$ \phi = \frac{s}{r} $$
where $s$ is a random integer between $0$ and $r-1$.  Finally, we can use a method called the continued fraction algorithm on $\phi$ in order to find r.  The final circuit will look something like this



<img src="./resources/Shor_circuit.png" alt="Short Circuit" style="width: 1000px;"/>


***

Below we'll provide the unitary $U$ needed for solving this period finding problem with $a=7$ and $N=15$

$$ 
    U\ket{y} = \ket{7y\text{mod}15}.
$$

To create $U^x$ we will simply repeat the circuit $x$ times.  The cell below will construct this unitary

In [ ]:
## Create 7mod15 gate
N = 15
m = int(np.ceil(np.log2(N)))

U_qc = qk.QuantumCircuit(m)
U_qc.x(range(m))
U_qc.swap(1, 2)
U_qc.swap(2, 3)
U_qc.swap(0, 3)

U = U_qc.to_gate()
U.name ='{}Mod{}'.format(7, N)

Confirm if the operator $U$ works as intended by creating a quantum circuit with $m$ qubits.  Prepare the inpute state representing any integer between $0$ and $15$ (remembering that Qiskit uses little endian notation) such as $\ket{1} = \ket{0001}$, $\ket{5} = \ket{0101}$, etc. and apply the $U$ gate to it.  Check if the circuit produces the expected outcomes for several inputs: $\ket{1}$, $\ket{3}$, and $\ket{5}$.  Run these circuits through the `aer_simulator` backend with $10000$ shots, save the count data as `input_1`, `input_3`, and `input_5`.

In [ ]:
### your code goes here

qcirc = qk.QuantumCircuit(m)




In [ ]:
## Run this cell to simulate 'qpe4' and to plot the histogram of the result
sim = qk.Aer.get_backend('aer_simulator')
shots = 20000
#count_qpe4 = qk.execute(qcirc, sim, shots=shots).result().get_counts()

input_1 = qk.execute(qcirc1, sim, shots=shots).result().get_counts()  # save the count data for input 1

input_3 = # save the count data for input 3
input_5 = # save the count data for input 5


In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex4

grade_lab3_ex4([input_1, input_3, input_5])

## Step 2. Implementing $U^{2^{m-1}}$

Now we'll use this controlled-$U$ to estimate the phase $\phi=\frac{s}{r}$.  First create a quantum circuit with $m=4$ qubits implementing the `7mod15` gate $2^2$ times and run it on the `unitary_simulator` backend to obtain the matrix represenation of the gates in the circuit.  Verify $U^2=I$.

In [ ]:

unitary_circ = qk.QuantumCircuit(m)

# Your code goes here



In [ ]:
sim = qk.Aer.get_backend('unitary_simulator')
unitary = qk.execute(unitary_circ, sim).result().get_unitary()

In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex5

grade_lab3_ex5(unitary)

## Step 3. Finding $\phi$ and Continued Fractions


Now armed with a way to execute $U^{2^{m-1}}$, let's use it in the QPE circuit you created earlier.  Use $8$ counting qubits and, using the `aer_simulator` again, estimate the phase $\phi$ given an input state of $\ket{1}$.

In [ ]:
# your code goes here

shor_qpe = #Create the QuantumCircuit needed to run with 8 counting qubits


In [ ]:
## Run this cell to simulate 'shor_qpe' and to plot the histogram of the results
sim = qk.Aer.get_backend('aer_simulator')
shots = 20000
shor_qpe_counts = qk.execute(shor_qpe, sim, shots=shots).result().get_counts()
plot_histogram(shor_qpe_counts, figsize=(9,5))

In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex6

grade_lab3_ex6(shor_qpe_counts)

We can then find the integers $s$ and $r$ using the continued fractions algorithm.  Luckily python has built-in functionality for this using the `Fraction` function, where we will limit the denominator to $r<15$.  Use this to find the estimated $s$ and $r$ for each outcome you measured above.

In [ ]:
from fractions import Fraction
print(Fraction(0.666), '\n')
print(Fraction(0.666).limit_denominator(15))

In [ ]:

shor_qpe_fractions = # create a list of Fraction objects for each measurement outcome


In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex7

grade_lab3_ex7(shor_qpe_fractions)

## Step 4. Putting it all together

Now let's put all of these steps together in order to factor the (very simple) number,  $N = 15$.  We'll continue with our example of $a=7$, remember that the phase we measure $s/r$ where $s$ is a random integer between $0$ and $r-1$ and:

$$
    a^r\text{mod}N = 1
$$

Then, once we have $r$, we can find a factor of $N$ by:

$$
    \left(a^r-1\right)\text{mod} N = 0
$$
which requires that $N$ must divide by $a^r-1$.  If $r$ is even, we can also write

$$
    a^r-1 = \left(a^{r/2}+1\right)\left(a^{r/2}-1\right).
$$

Put together a function called `shor_qpe` which takes arguments for $N$, and $m$ (the number of counting qubits) and composes, runs, and processes Shor's algorithm to guess the factors.

In [ ]:
def shor_qpe(N, m):

    return None

In [ ]:
# Run this on Aer_simulator

In [ ]:
from qc_grader.challenges.qgss_2023 import grade_lab3_ex8

grade_lab3_ex8(shor_qpe)

Congratulations! You've completed Lab 3 of the Global Summer School!! 🎉

This lab was adapted from both the [Qiskit QPE Lab](https://learn.qiskit.org/course/ch-labs/lab-5-accuracy-of-quantum-phase-estimation#lab-3-0) as well as the [Qiskit Shor's Algorithm](https://learn.qiskit.org/course/ch-labs/lab-7-scalable-shors-algorithm) lab